# Import libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os
import time
from setup_dataset import *

# Load matches

In [2]:
def get_actions_list(zigzag=False):

    if zigzag:

        ACTIONS = {
            "right": 2,
            "left": 3,
        }

    else:

        ACTIONS = {
            "noop": 0,
            "accelerate": 1,
            "right": 2,
            "left": 3,
            "break": 4,
            "right_break": 5,
            "left_break": 6,
            "right_accelerate": 7,
            "left_accelerate": 8,
        }

    return list(ACTIONS.values())

In [3]:
def set_device():

    use_gpu = input(("Do you want to use GPU (y/n)"))

    if use_gpu == 'y' and torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        print("Selected CPU")

    return device

In [4]:
data_info = {
    'datapath' : r"../1-generate/data/",
    'available_targets' : get_actions_list(zigzag=False),
    'match_list' : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'start_frame' : 1,
    'end_frame' : 4459,
    'device' : set_device(),
}

Do you want to use GPU (y/n)n
Selected CPU


In [5]:
class dataset():

    def __init__(self, data_info):
        self.datapath = data_info['datapath']
        self.available_targets = data_info['available_targets']
        self.match_list = data_info['match_list']
        self.start_frame = data_info['start_frame']
        self.end_frame = data_info['end_frame']
        self.device = data_info['device']
        self.data, self.targets = self.setDataFromMatch()
        self.target_size = self.targets.shape
        self.seq_len = self.setSeqLen(self.data)
    
    def setSeqLen(self, X_train):
        return torch.FloatTensor(list(map(len,X_train)))

    def setDataFromMatch(self):
        frames_arr = []
        actions_arr = []

        for m in self.match_list:
            
            _, frames, actions, _, _ = self.load_npz(self.datapath, m)
            frames = frames[self.start_frame - 1:self.end_frame, 30:130, :]
            actions = actions[self.start_frame - 1:self.end_frame]
            
            action_one_hot = [self.prepare_action_data(i, self.available_targets) for i in actions]
            actions = np.array(action_one_hot)
            actions = actions.reshape(len(actions), -1)
            
            frames_arr.append(frames)
            actions_arr.append(actions)

        frames_arr = np.array(frames_arr)
        actions_arr = np.array(actions_arr)

        return frames_arr, actions_arr
    
    def normalize_images(self, frames):
        return frames/255

    def load_npz(self, data_path, m):
        
        path = data_path + "match_" + str(m) + "/npz/"

        actions = np.load(path + 'actions.npz')
        #lifes = np.load(path + 'lifes.npz')
        frames = np.load(path + 'frames.npz')
        rewards = np.load(path + 'rewards.npz')

        arr_actions = actions.f.arr_0
        # arr_lifes = lifes.f.arr_0
        arr_lifes = np.array([])
        arr_frames = frames.f.arr_0
        arr_rewards = rewards.f.arr_0

        print("Successfully loaded NPZ.")

        return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

    def prepare_action_data(self, action, ACTIONS_LIST):

        new_action = np.zeros((1, len(ACTIONS_LIST)), dtype=int) 

        new_action[0, ACTIONS_LIST.index(action)] = 1

        return new_action

In [6]:
data = dataset(data_info)

Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.


In [7]:
data.seq_len

tensor([4459., 4459., 4459., 4459., 4459., 4459., 4459., 4459., 4459., 4459.])

In [8]:
data.data = torch.tensor(data.data)
data.targets = torch.tensor(data.targets)

In [9]:
data.data = data.data.view(data.data.shape[0], data.data.shape[1], -1)

In [10]:
data.data = data.data/255

In [11]:
"""
data = chunk_dataset(match_list=[1,2,3,4,5,6,7,8],
                     start_frame = 1,
                     end_frame = 1020
                    )
"""

'\ndata = chunk_dataset(match_list=[1,2,3,4,5,6,7,8],\n                     start_frame = 1,\n                     end_frame = 1020\n                    )\n'

In [12]:
def padSequence(raw):
    return pad_sequence(raw, batch_first=True).float()

In [13]:
def packPaddedSequence(seq, seq_len):
    return pack_padded_sequence(seq, seq_len, batch_first=True, enforce_sorted=False)

Prepare frames

In [14]:
"""data.data = padSequence(data.data)"""

'data.data = padSequence(data.data)'

In [15]:
"""padded_shape = data.data.shape"""

'padded_shape = data.data.shape'

In [16]:
"""data.data = packPaddedSequence(data.data, data.seq_len)"""

'data.data = packPaddedSequence(data.data, data.seq_len)'

Prepare targets

In [17]:
"""data.targets = padSequence(data.targets)"""

'data.targets = padSequence(data.targets)'

# Create model

In [18]:
def set_type_name():
    print("Choose type of RNN model:")
    print("1 - Simple RNN")
    print("2 - LSTM")
    print("3 - CNN")
    choise = int(input("type: "))
    if choise == 1:
        return "RNN"
    elif choise == 2:
        return "LSTM"
    elif choise == 3:
        return "CNN"
    else:
        return "Error"

In [19]:
model_info = {
    'input_size' : 12000,
    'hidden_neurons' : int(input("Number of hidden neurons: ")),
    'output_size' : 9,
    'n_epochs' : int(input("Number of epochs: ")),
    'min_loss' : 1e-5,
    'n_layers' : 1,
    'type' : set_type_name(),
}

Number of hidden neurons: 200
Number of epochs: 10000
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 2


In [20]:
class LSTMModel(nn.Module):
    def __init__(self, data_info, model_info):
        super(LSTMModel, self).__init__()
        
        self.device = data_info['device']

        # Defining some parameters
        self.input_size = model_info['input_size']
        self.hidden_dim = model_info['hidden_neurons']
        self.n_layers = model_info['n_layers']
        self.output_size = model_info['output_size']

        self.init_hidden()
        self.lstm = nn.LSTM(self.input_size, self.hidden_dim, self.n_layers, batch_first=True)  
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
        
        self.out = nn.Softmax()
    
    def forward(self, x):
        
        hidden = self.init_hidden()

        pad_embed_pack_lstm = self.lstm(x, hidden)
        pad_embed_pack_lstm_pad = pad_packed_sequence(pad_embed_pack_lstm[0], batch_first=True)
        
        outs, _ = pad_embed_pack_lstm_pad
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = outs.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        out = self.out(out)
        
        return out

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(1, self.input_size, self.hidden_dim)
        hidden_b = torch.randn(1, self.input_size, self.hidden_dim)

        if self.device.type == 'cuda':
            hidden_a = hidden_a.cuda()
            hidden_b = hidden_b.cuda()

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

In [21]:
def set_model(model_info):

    if model_info['type'] == "RNN":
        model = RNNModel(device=model_structure.device, 
                         input_size=model_structure.data_size, 
                         output_size=model_structure.output_size, 
                         hidden_dim=model_structure.hidden_neurons, 
                         n_layers=1)
    elif model_info['type'] == "LSTM":
        model = LSTMModel(data_info, model_info)
    elif model_info['type'] == "CNN":
        model = CNNLSTMModel(device=model_structure.device, 
                             input_size=model_structure.data_size, 
                             output_size=model_structure.output_size, 
                             hidden_dim=model_structure.hidden_neurons, 
                             n_layers=1)
    else:
        print("ERROR defining Model")

    return model

In [22]:
criterion = nn.MSELoss()
model = set_model(model_info)
optimizer = torch.optim.Adam(model.parameters())

# Prepare to store

In [23]:
def setName(data_info, model_info):
    x = [str(num) for num in data_info['match_list']]
    x = '-'.join(x)
    obs = input("write a observations without space and punctuations:")
    name = f"{model_info['type']}_{obs}_m{x}_f{data_info['start_frame']}to{data_info['end_frame']}_epoch{model_info['n_epochs']}_h{model_info['hidden_neurons']}"
    return name

In [24]:
def setPath(data_info, model_info):
    name = setName(data_info, model_info)
    newpath = f"models/" + name
    if not os.path.exists(newpath):
        print(f"models/" + name + " created")
        os.makedirs(newpath)
    else:
        print(f"models/" + name)
        print("ATTENTION! folder not created. Training informations will overwrite the existing one")
    return newpath

In [25]:
model_path = setPath(data_info, model_info)

write a observations without space and punctuations:teste
models/LSTM_teste_m2-3-4-5-6-7-8-9-10-11_f1to4459_epoch10000_h200 created


# Train model

In [26]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    correct = torch.sum(predicted == target)

    acc = correct/predicted.shape[0]
    return float(acc)

In [27]:
if data_info['device'].type == 'cuda':
    mymodel = model.cuda()
    X_train = data.data.cuda() 
    Y_train = data.targets.cuda()
else:
    mymodel = model
    X_train = data.data 
    Y_train = data.targets

In [28]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])
valid_acc_arr = np.array([])

In [29]:
def create_info_string(current_epoch, last_epoch, train_loss, train_acc, valid_acc):
    epoch_info = f"Epoch: {current_epoch}/{last_epoch}-------------------------------------------"
    train_info = f"Train -> Loss: {train_loss:.15f} Acc: {train_acc:.15f}"
    valid_info = f"Valid -> Acc: {valid_acc:.15f}"
    return epoch_info, train_info, valid_info

In [30]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=len(data.data), shuffle=False)

In [31]:
X_train.shape

torch.Size([10, 4459, 12000])

In [ ]:
start_time_processing = time.time()

loss_file = open(model_path + '/' + "loss_file.txt", "w")

best_loss = 1
first_epoch = True

for fold, (train_ids, test_ids) in enumerate(kfold.split(data.data)):
    
    print(f'Fold: {fold}')

    X_train = data.data[train_ids]
    Y_train = data.targets[train_ids]
    X_valid = data.data[test_ids]
    Y_valid = data.targets[test_ids]
    
    X_train = padSequence(X_train)
    X_valid = padSequence(X_valid)
    
    X_train_padded_shape = X_train.shape
    X_train_seq_len = data.seq_len[train_ids]
    X_train = packPaddedSequence(X_train, X_train_seq_len)
    
    X_valid_padded_shape = X_valid.shape
    X_valid_seq_len = data.seq_len[test_ids]
    X_valid = packPaddedSequence(X_valid, X_valid_seq_len)
    
    if data_info['device'].type == 'cuda':
        mymodel = model.cuda()
        X_train = X_train.cuda() 
        Y_train = Y_train.cuda()
        X_valid = X_valid.cuda() 
        Y_valid = Y_valid.cuda()
    
    for epoch in range(1, model_info['n_epochs'] + 1):

        mymodel.train()

        optimizer.zero_grad()
        output = mymodel(X_train)

        loss = criterion(output, Y_train.view(-1,model_info['output_size']).float())
        loss.backward()
        optimizer.step()

        if epoch%10 == 0:

            train_loss_arr = np.append(train_loss_arr, loss.item())
            train_acc_arr  = np.append(train_acc_arr, get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
            
            mymodel.eval()
            
            output = mymodel(X_valid)
            valid_acc_arr  = np.append(valid_acc_arr, get_acc(output, Y_valid.reshape(-1, len(data.available_targets))))
                        
            epoch_info, train_info, valid_info = create_info_string(epoch, 
                                                                    model_info['n_epochs'], 
                                                                    train_loss_arr[-1], 
                                                                    train_acc_arr[-1],
                                                                    valid_acc_arr[-1])
            loss_file.write(epoch_info)
            loss_file.write(train_info)
            loss_file.write(valid_info)
            print(epoch_info)
            print(train_info)
            print(valid_info)

            if train_loss_arr[-1] < best_loss:
                state = {'epoch': epoch, 
                         'state_dict': mymodel.state_dict(),
                         'optimizer': optimizer.state_dict(), 
                         'losslogger': loss.item(), 
                        }
                torch.save(state, model_path + '/' + f'model_fold_{fold}')
                best_loss = loss.item()
            else:
                print("model not saved")
                
time_info = "--- %s seconds ---" % (time.time() - start_time_processing)
loss_file.write(time_info)
print(time_info)

loss_file.close()
np.savez(model_path + '/' + "train_loss_arr", train_loss_arr)

Fold: 0


C:\Users\alece\miniconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 10/10000-------------------------------------------
Train -> Loss: 0.071175277233124 Acc: 0.549849271774292
Valid -> Acc: 0.521417379379272
Epoch: 20/10000-------------------------------------------
Train -> Loss: 0.069816969335079 Acc: 0.549849271774292
Valid -> Acc: 0.521641612052917
Epoch: 30/10000-------------------------------------------
Train -> Loss: 0.068166777491570 Acc: 0.564177334308624
Valid -> Acc: 0.528818130493164
Epoch: 40/10000-------------------------------------------
Train -> Loss: 0.066141732037067 Acc: 0.579228043556213
Valid -> Acc: 0.530612230300903
Epoch: 50/10000-------------------------------------------
Train -> Loss: 0.064387656748295 Acc: 0.593381702899933
Valid -> Acc: 0.535546064376831
Epoch: 60/10000-------------------------------------------
Train -> Loss: 0.062130432575941 Acc: 0.606762826442719
Valid -> Acc: 0.538012981414795
Epoch: 70/10000-------------------------------------------
Train -> Loss: 0.059906572103500 Acc: 0.626124441623688
Val

In [ ]:
"""start_time_processing = time.time()

loss_file = open(model_path + '/' + "loss_file.txt", "w")

best_loss = 1
first_epoch = True

for fold, (train_ids, test_ids) in enumerate(kfold.split(data.data)):
    
    print(f'Fold: {fold}')

    X_train = data.data[train_ids]
    Y_train = data.targets[train_ids]
    X_valid = data.data[test_ids]
    Y_valid = data.targets[test_ids]
    
    X_train = padSequence(X_train)
    X_valid = padSequence(X_valid)
    
    X_valid_padded_shape = X_valid.shape
    X_valid_seq_len = data.seq_len[test_ids]
    X_valid = packPaddedSequence(X_valid, X_valid_seq_len)
    
    if data_info['device'].type == 'cuda':
        mymodel = model.cuda()
        X_train = X_train.cuda() 
        Y_train = Y_train.cuda()
        X_valid = X_valid.cuda() 
        Y_valid = Y_valid.cuda()
    
    for epoch in range(1, model_info['n_epochs'] + 1):
        
        print("epoch:", epoch)
        
        mymodel.train()
            
        for sequence, targets, seq_len2 in zip(X_train, Y_train, data.seq_len[train_ids]):
            
            print(sequence.shape)
            print(targets.shape)
            
            sequence = sequence.view(1, sequence.shape[0], sequence.shape[1])
            X_train_padded_shape = sequence.shape
            X_train_seq_len = torch.tensor([seq_len2])
            sequence = packPaddedSequence(sequence, X_train_seq_len)

            optimizer.zero_grad()
            output = mymodel(sequence)

            loss = criterion(output, targets.view(-1,model_info['output_size']).float())
            loss.backward()
            optimizer.step()

        if epoch%10 == 0:

            train_loss_arr = np.append(train_loss_arr, loss.item())
            train_acc_arr  = np.append(train_acc_arr, get_acc(output, sequence.reshape(-1, len(data.available_targets))))
            
            mymodel.eval()
            
            output = mymodel(X_valid)
            valid_acc_arr  = np.append(valid_acc_arr, get_acc(output, Y_valid.reshape(-1, len(data.available_targets))))
                        
            epoch_info, train_info, valid_info = create_info_string(epoch, 
                                                                    model_info['n_epochs'], 
                                                                    train_loss_arr[-1], 
                                                                    train_acc_arr[-1],
                                                                    valid_acc_arr[-1])
            loss_file.write(epoch_info)
            loss_file.write(train_info)
            loss_file.write(valid_info)
            print(epoch_info)
            print(train_info)
            print(valid_info)

            if train_loss_arr[-1] < best_loss:
                state = {'epoch': epoch, 
                         'state_dict': mymodel.state_dict(),
                         'optimizer': optimizer.state_dict(), 
                         'losslogger': loss.item(), 
                        }
                torch.save(state, model_path + '/' + f'model_fold_{fold}')
                best_loss = loss.item()
            else:
                print("model not saved")
                
time_info = "--- %s seconds ---" % (time.time() - start_time_processing)
loss_file.write(time_info)
print(time_info)

loss_file.close()
np.savez(model_path + '/' + "train_loss_arr", train_loss_arr)"""